# **<span style="color:#1E90FF">Task 1 - Data Extracting (Scraping) :</span>**
---

**<h2 style="color: #00008B;">A. Use IMDB website:</h2>**


**• You are required to extract data from the IMDB** (_Internet Movie Database_) **, Website:
https://www.imdb.com/chart/top/**

**• This page contains links to Top 250 Movies.**

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time , re

In [2]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}
BASE_URL = 'https://www.imdb.com'

In [3]:
response = requests.get(f"{BASE_URL}/chart/top/", headers=HEADERS)
soup = BeautifulSoup(response.text, 'html.parser')
movie_elements = soup.select('li.ipc-metadata-list-summary-item')[:25]

______________________________________________________________________________

**<h2 style="color: #00008B;">B. Define Data to Scrape :</h2>**

<h3 style="color: #00008B;">1. <code>movies</code> DataFrame</h3>
<p>Create a table named <code>movies</code> to scrape data from IMDb's "Top 250 Movies" page.</p>

<b style="color: #00008B;">Basic Movie Information:</b>
<ul style="color: #00008B;">
  <li>Title</li>
  <li>Release Year</li>
  <li>Rating</li>
  <li>Director(s)</li>
  <li>Genre(s)</li>
</ul>

<b style="color: #00008B;">Additional Metadata:</b>
<ul style="color: #00008B;">
  <li>Release Date</li>
  <li>Country of Origin</li>
  <li>Official Sites</li>
  <li>Languages</li>
  <li>Filming Locations</li>
  <li>Production Companies</li>
  <li>Budget</li>
  <li>Gross Worldwide Revenue</li>
</ul>

<hr>

<h3 style="color: #00008B;">2. <code>full_cast_and_crew</code> DataFrame</h3>
<p>For each movie, retrieve full information on the cast and "Produced by" members from the corresponding movie page on IMDb.</p>

<b style="color: #00008B;">Table Structure for Cast and Crew:</b>
<ul style="color: #00008B;">
  <li><code>Movie_ID</code>: Unique identifier for each movie, used to link with the <code>movies</code> table</li>
  <li><code>Name</code>: The name of each cast member or "Produced by" member</li>
  <li><code>Role_Type</code>: Specifies the type of role (e.g., Director, Actor, Producer)</li>
</ul>


In [7]:
print("Loading the list of top 250 movies...")
all_data = []
cast_crew_data = []

for movie in tqdm(movie_elements):  
    title_tag = movie.select_one('h3.ipc-title__text')
    title = title_tag.text.split('. ')[1] if title_tag else "N/A"
    
    year_tag = movie.select_one('span.sc-14dd939d-6') or movie.select_one('span.cli-title-metadata-item')
    year = year_tag.get_text(strip=True) if year_tag else "N/A"
    
    rating_tag = movie.select_one('span.ipc-rating-star--imdb')
    rating = rating_tag.text.split()[0] if rating_tag else "N/A"
    
    link_tag = movie.select_one('a.ipc-title-link-wrapper')
    movie_link = BASE_URL + link_tag['href'] if link_tag else BASE_URL
    
    movie_id = movie_link.split('/')[-2]
    
#_____________________________________________________________________________
                        
    additional_metadata=[]
    budget_tag = soup.find('li', {'data-testid': 'title-boxoffice-budget'})
    if budget_tag:
        raw_budget = budget_tag.get_text(strip=True)
        match = re.search(r'\$[\d,]+', raw_budget)
        if match:
            budget = match.group(0)
        else:
            budget = "N/A"
    else:
        budget = "N/A"
    #_____________________________________

    gross_revenue_tag = soup.find('li', {'data-testid': 'title-boxoffice-cumulativeworldwidegross'})
    if gross_revenue_tag:
        raw_gross_revenue = gross_revenue_tag.get_text(strip=True)
        match = re.search(r'\$[\d,]+', raw_gross_revenue)
        if match:
            gross_revenue = match.group(0)
        else:
            gross_revenue = "N/A"
    else:
        gross_revenue = "N/A"
    #_____________________________________

    official_sites_tag = soup.find('li', {'data-testid': 'details-officialsites'})
    official_sites = official_sites_tag.get_text(strip=True) if official_sites_tag else "N/A"
    #_____________________________________

    release_date_tag = soup.find('a', {'title': 'See more release dates'})
    release_date = release_date_tag.get_text(strip=True) if release_date_tag else "N/A"

    text = soup.get_text()
    release_date_match = re.search(r'\b\w+\s\d{1,2},\s\d{4}\b', text)
    if release_date_match:
        release_date = release_date_match.group(0)
        release_date = re.sub(r'^date', '', release_date) 
    else:
        release_date = "N/A"
    #_____________________________________

    country_tag = soup.find('li', {'data-testid': 'title-details-origin'})
    if country_tag:
        country_links = country_tag.find_all('a')
        countries = [link.get_text(strip=True).replace(u'\xa0', ' ') for link in country_links]
        country_of_origin = ', '.join(countries) if countries else 'N/A'
    else:
        country_of_origin = 'N/A'  
    #_____________________________________

    genres_tag = soup.find_all('span', {'class': 'ipc-chip__text'})
    if genres_tag:
        genres = [genre.get_text(strip=True) for genre in genres_tag[:-1]]  
    else:
        genres ='N/A'
    #_____________________________________

    language_section = soup.find('li', {'data-testid': 'title-details-languages'})
    if language_section:
        languages = [lang.get_text(strip=True) for lang in language_section.find_all('a')]
    else:
        languages = "N/A"
    #_____________________________________

    filming_location_tag = soup.find('li', {'data-testid': 'title-details-filminglocations'})
    if filming_location_tag:
        filming_location_text = filming_location_tag.get_text(strip=True)
        filming_location_text = filming_location_text.replace(u'\xa0', ' ').strip()
        filming_location = filming_location_text.replace('Filming locations', '', 1).strip()
        filming_location = filming_location if filming_location else 'N/A'
    else:
        filming_location = 'N/A' 
    #_____________________________________
    production_companies_tag = soup.find('a', {'href': re.compile(r'/company/')})
    if production_companies_tag:
        production_companies = production_companies_tag.get_text(strip=True).replace('\u00A0', ' ') if production_companies_tag else "N/A"
    else:
        production_companies = 'N/A'
#_____________________________________________________________________________
    
    all_data.append({
                'Movie_ID': movie_id,
                'Title': title,
                'Year': year,
                'Rating': rating,
                'Link': movie_link,
                'Budget' : budget,
                'gross_revenue' : gross_revenue,
                'official_sites' : official_sites,
                'release_date' :release_date,
                'languages' :languages,
                'genres':genres,
                'production_companies':production_companies,
                'filming_location': filming_location,
                'country_of_origin':country_of_origin
            }) 
# _____________________________________________________________________________

    response = requests.get(movie_link, headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')
        
#_____________________________________________________________________________

    cast_section = soup.find('section', {'data-testid': 'title-cast'}) 
    if cast_section:
        cast_items = cast_section.find_all('div', {'data-testid': 'title-cast-item'}) 
        for item in cast_items:
            actor = item.find('a', {'data-testid': 'title-cast-item__actor'})
            character = item.find('a', {'data-testid': 'cast-item-characters-link'}) or \
                        item.find('span', {'data-testid': 'cast-item-characters-link'})
                
            if actor:
                cast_crew_data.append({
                    'Movie_ID': movie_id,
                    'Name': actor.get_text(strip=True),
                    'Role_Type': 'Actor',
                    'Character_Position': character.get_text(strip=True) if character else 'N/A'
                })
    
#_____________________________________________________________________________

    directors_section = soup.find('meta', {'name': 'description'})  
    if directors_section:
        description = directors_section.get('content', '') 
        directors_match = re.search(r'Directed by (.+?)\. With', description)
        if directors_match:
            directors = directors_match.group(1) 
            cast_crew_data.append({
                'Movie_ID': movie_id,
                'Name': directors,
                'Role_Type': 'Director',
                'Character_Position': 'N/A'
            })
#_____________________________________________________________________________
  
    time.sleep(3)
    

Loading the list of top 250 movies...


100%|██████████| 25/25 [02:24<00:00,  5.78s/it]


In [8]:
df = pd.DataFrame(all_data)
cast_crew_data = pd.DataFrame(cast_crew_data)
df.to_csv('imdb_complete_data.csv', index=False, encoding='utf-8-sig')
cast_crew_data.to_csv('cast_crew_data.csv', index=False, encoding='utf-8-sig')

print("All data has been successfully saved to the file imdb_complete_data.csv")
print("\nShow sample data:")

All data has been successfully saved to the file imdb_complete_data.csv

Show sample data:


In [9]:
df.head()

,Movie_ID,Title,Year,Rating,Link,Budget,gross_revenue,official_sites,release_date,languages,genres,production_companies,filming_location,country_of_origin
0,tt0111161,The Shawshank Redemption,1994,9.3,https://www.imdb.com/title/tt0111161/?ref_=cht...,"$3,300,000","$30,681,033",Official sitesOfficial FacebookOfficial Site,"February 13, 2004",[Portuguese],"[Caper, Coming-of-Age, Gangster, Crime, Drama]",O2 Filmes,"Rio de Janeiro, Rio de Janeiro, Brazil","Brazil, France"
1,tt0068646,The Godfather,1972,9.2,https://www.imdb.com/title/tt0068646/?ref_=cht...,"$25,000,000","$29,332,133",Official sitesOfficial FacebookWarner Bros. (U...,"October 14, 1994",[English],"[Epic, Period Drama, Prison Drama, Drama]",Castle Rock Entertainment,"Mansfield Reformatory - 100 Reformatory Road, ...",United States
2,tt0468569,The Dark Knight,2008,9.0,https://www.imdb.com/title/tt0468569/?ref_=cht...,"$6,000,000","$250,342,198",Official sitesFacebookInstagram,"March 24, 1972","[English, Italian, Latin]","[Epic, Gangster, Tragedy, Crime, Drama]",Paramount Pictures,"Forza d'Agrò, Messina, Sicily, Italy(as Corleo...",United States
3,tt0071562,The Godfather Part II,1974,9.0,https://www.imdb.com/title/tt0071562/?ref_=cht...,"$185,000,000","$1,009,057,329",Official sitesFacebookOfficial Site,"July 18, 2008","[English, Mandarin]","[Action Epic, Epic, Superhero, Tragedy, Action...",Warner Bros.,"Chicago, Illinois, USA","United States, United Kingdom"
4,tt0050083,12 Angry Men,1957,9.0,https://www.imdb.com/title/tt0050083/?ref_=cht...,"$13,000,000","$47,980,865",Official sitesFacebookInstagram,"December 18, 1974","[English, Italian, Spanish, Latin, Sicilian]","[Epic, Gangster, Tragedy, Crime, Drama]",Paramount Pictures,"Kaiser Estate - 4000 W Lake Blvd, Homewood, La...",United States


In [10]:
cast_crew_data.head()

,Movie_ID,Name,Role_Type,Character_Position
0,tt0111161,Tim Robbins,Actor,Andy Dufresne
1,tt0111161,Morgan Freeman,Actor,Ellis Boyd 'Red' Redding
2,tt0111161,Bob Gunton,Actor,Warden Norton
3,tt0111161,William Sadler,Actor,Heywood
4,tt0111161,Clancy Brown,Actor,Captain Hadley


 # **<span style="color:#1E90FF">Task 2 - Data Preprocessing and Transformation (Scraping) :</span>**

**<h2 style="color: #00008B;">C. Clean the Scraped Data :</h2>**

<h3 style="color: #00008B;">1. Perform at Least 3 Cleaning Tasks:</h3>
<ul style="color: #00008B;">
  <li>Handle missing values</li>
  <li>Remove duplicate entries</li>
  <li>Correct inconsistencies in formatting (e.g., date formats, case sensitivity)</li>
</ul>


In [11]:
print("Data cleaning process has started")
df = pd.read_csv('imdb_complete_data.csv')
cast_crew_data = pd.read_csv('cast_crew_data.csv')

Data cleaning process has started


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Movie_ID              25 non-null     object 
 1   Title                 25 non-null     object 
 2   Year                  25 non-null     int64  
 3   Rating                25 non-null     float64
 4   Link                  25 non-null     object 
 5   Budget                24 non-null     object 
 6   gross_revenue         25 non-null     object 
 7   official_sites        21 non-null     object 
 8   release_date          25 non-null     object 
 9   languages             25 non-null     object 
 10  genres                25 non-null     object 
 11  production_companies  25 non-null     object 
 12  filming_location      25 non-null     object 
 13  country_of_origin     25 non-null     object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.9+ KB


In [13]:
cast_crew_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Movie_ID            475 non-null    object
 1   Name                475 non-null    object
 2   Role_Type           475 non-null    object
 3   Character_Position  450 non-null    object
dtypes: object(4)
memory usage: 15.0+ KB


In [14]:
df.isnull().sum()

Movie_ID                0
Title                   0
Year                    0
Rating                  0
Link                    0
Budget                  1
gross_revenue           0
official_sites          4
release_date            0
languages               0
genres                  0
production_companies    0
filming_location        0
country_of_origin       0
dtype: int64

In [15]:
cast_crew_data.isnull().sum()

Movie_ID               0
Name                   0
Role_Type              0
Character_Position    25
dtype: int64

In [16]:
df['Budget'] = df['Budget'].replace('[\$,]', '', regex=True) 
df['Budget'] = pd.to_numeric(df['Budget'], errors='coerce') 

In [17]:
df['Budget'] = df['Budget'].fillna(df['Budget'].mean())

In [18]:
df['gross_revenue'] = df['gross_revenue'].replace('[\$,]', '', regex=True) 
df['gross_revenue'] = pd.to_numeric(df['gross_revenue'], errors='coerce')  

In [19]:
df['gross_revenue'] = df['gross_revenue'].fillna(df['gross_revenue'].mean())

In [20]:
df.fillna({
    'official_sites': 'Not Available',
    'release_date':'Not Available',
    'production_companies':'Not Available',
    'filming_location':'Not Available',
    'country_of_origin':'Not Available'
}, inplace=True)

In [21]:
cast_crew_data.fillna({
    'Character_Position': 'Not Available',
}, inplace=True)

In [22]:
df.isnull().sum()

Movie_ID                0
Title                   0
Year                    0
Rating                  0
Link                    0
Budget                  0
gross_revenue           0
official_sites          0
release_date            0
languages               0
genres                  0
production_companies    0
filming_location        0
country_of_origin       0
dtype: int64

In [23]:
cast_crew_data.isnull().sum()

Movie_ID              0
Name                  0
Role_Type             0
Character_Position    0
dtype: int64

In [24]:
df.duplicated().sum()

0

In [25]:
cast_crew_data.duplicated().sum() 

0

**<h2 style="color: #00008B;">D. Transform the Data :</h2>**

<h3 style="color: #00008B;">1. Perform at Least 3 Transformation Tasks:</h3>
<ul style="color: #00008B;">
  <li>Normalize, standardize, or encode categorical variables as needed</li>
  <li>Convert columns into formats suitable for MySQL (e.g., format date columns, encode text values)</li>
  <li>Integrate the <code>full_cast_and_crew</code> table with the main <code>movies</code> table using <code>Movie_ID</code> as a foreign key</li>
</ul>

In [27]:
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].map(lambda x: str(x).strip().lower() if pd.notnull(x) else x)

In [28]:
categorical_cols = cast_crew_data.select_dtypes(include=['object']).columns
cast_crew_data[categorical_cols] = cast_crew_data[categorical_cols].apply(lambda col: col.str.lower() if col.notnull().any() else col)

In [29]:
df.head()

,Movie_ID,Title,Year,Rating,Link,Budget,gross_revenue,official_sites,release_date,languages,genres,production_companies,filming_location,country_of_origin
0,tt0111161,the shawshank redemption,1994,9.3,https://www.imdb.com/title/tt0111161/?ref_=cht...,3300000.0,30681033,official sitesofficial facebookofficial site,"february 13, 2004",['portuguese'],"['caper', 'coming-of-age', 'gangster', 'crime'...",o2 filmes,"rio de janeiro, rio de janeiro, brazil","brazil, france"
1,tt0068646,the godfather,1972,9.2,https://www.imdb.com/title/tt0068646/?ref_=cht...,25000000.0,29332133,official sitesofficial facebookwarner bros. (u...,"october 14, 1994",['english'],"['epic', 'period drama', 'prison drama', 'drama']",castle rock entertainment,"mansfield reformatory - 100 reformatory road, ...",united states
2,tt0468569,the dark knight,2008,9.0,https://www.imdb.com/title/tt0468569/?ref_=cht...,6000000.0,250342198,official sitesfacebookinstagram,"march 24, 1972","['english', 'italian', 'latin']","['epic', 'gangster', 'tragedy', 'crime', 'drama']",paramount pictures,"forza d'agrò, messina, sicily, italy(as corleo...",united states
3,tt0071562,the godfather part ii,1974,9.0,https://www.imdb.com/title/tt0071562/?ref_=cht...,185000000.0,1009057329,official sitesfacebookofficial site,"july 18, 2008","['english', 'mandarin']","['action epic', 'epic', 'superhero', 'tragedy'...",warner bros.,"chicago, illinois, usa","united states, united kingdom"
4,tt0050083,12 angry men,1957,9.0,https://www.imdb.com/title/tt0050083/?ref_=cht...,13000000.0,47980865,official sitesfacebookinstagram,"december 18, 1974","['english', 'italian', 'spanish', 'latin', 'si...","['epic', 'gangster', 'tragedy', 'crime', 'drama']",paramount pictures,"kaiser estate - 4000 w lake blvd, homewood, la...",united states


In [30]:
cast_crew_data.head()

,Movie_ID,Name,Role_Type,Character_Position
0,tt0111161,tim robbins,actor,andy dufresne
1,tt0111161,morgan freeman,actor,ellis boyd 'red' redding
2,tt0111161,bob gunton,actor,warden norton
3,tt0111161,william sadler,actor,heywood
4,tt0111161,clancy brown,actor,captain hadley


In [31]:
df['Year'] = pd.to_numeric(df['Year'], errors='coerce').fillna(0).astype(int)
df['release_date'] = pd.to_datetime(df['release_date'], format='%d-%m-%y', errors='coerce').dt.date

In [42]:
df['Movie_ID'] = df['Movie_ID'].str.strip().str.upper()
cast_crew_data['Movie_ID'] = cast_crew_data['Movie_ID'].str.strip().str.upper()

cast_crew_data = cast_crew_data[cast_crew_data['Movie_ID'].isin(df['Movie_ID'])]

merged_df = pd.merge(cast_crew_data, df, on='Movie_ID', how='inner')

print("\nFirst 5 rows of Merged dataset (merged_df):")
merged_df.head()


First 5 rows of Merged dataset (merged_df):


,Movie_ID,Name,Role_Type,Character_Position,Title,Year,Rating,Link,Budget,gross_revenue,official_sites,release_date,languages,genres,production_companies,filming_location,country_of_origin
0,TT0111161,tim robbins,actor,andy dufresne,the shawshank redemption,1994,9.3,https://www.imdb.com/title/tt0111161/?ref_=cht...,3300000.0,30681033,official sitesofficial facebookofficial site,NaT,['portuguese'],"['caper', 'coming-of-age', 'gangster', 'crime'...",o2 filmes,"rio de janeiro, rio de janeiro, brazil","brazil, france"
1,TT0111161,morgan freeman,actor,ellis boyd 'red' redding,the shawshank redemption,1994,9.3,https://www.imdb.com/title/tt0111161/?ref_=cht...,3300000.0,30681033,official sitesofficial facebookofficial site,NaT,['portuguese'],"['caper', 'coming-of-age', 'gangster', 'crime'...",o2 filmes,"rio de janeiro, rio de janeiro, brazil","brazil, france"
2,TT0111161,bob gunton,actor,warden norton,the shawshank redemption,1994,9.3,https://www.imdb.com/title/tt0111161/?ref_=cht...,3300000.0,30681033,official sitesofficial facebookofficial site,NaT,['portuguese'],"['caper', 'coming-of-age', 'gangster', 'crime'...",o2 filmes,"rio de janeiro, rio de janeiro, brazil","brazil, france"
3,TT0111161,william sadler,actor,heywood,the shawshank redemption,1994,9.3,https://www.imdb.com/title/tt0111161/?ref_=cht...,3300000.0,30681033,official sitesofficial facebookofficial site,NaT,['portuguese'],"['caper', 'coming-of-age', 'gangster', 'crime'...",o2 filmes,"rio de janeiro, rio de janeiro, brazil","brazil, france"
4,TT0111161,clancy brown,actor,captain hadley,the shawshank redemption,1994,9.3,https://www.imdb.com/title/tt0111161/?ref_=cht...,3300000.0,30681033,official sitesofficial facebookofficial site,NaT,['portuguese'],"['caper', 'coming-of-age', 'gangster', 'crime'...",o2 filmes,"rio de janeiro, rio de janeiro, brazil","brazil, france"


In [44]:
merged_df.to_csv('imdb_complete_data_cleaned.csv', index=False, encoding='utf-8-sig')
print("The data has been saved in: imdb_complete_data_cleaned.csv")

The data has been saved in: imdb_complete_data_cleaned.csv


# **<span style="color:#1E90FF">Task 3: Loading Data and Executing Queries in MySQL Database Server  :</span>**
---

**<h2 style="color: #00008B;">A. Set Up a Local MySQL Database:</h2>**

<h3 style="color: #00008B;">1. Set Up a Local MySQL Database:</h3>
<ul style="color: #00008B;">
  <li>Install and set up a MySQL database server on your localhost</li>
</ul>

**<h2 style="color: #00008B;">B. Connect to the MySQL Database:</h2>**

<h3 style="color: #00008B;">2. Connect to the MySQL Database:</h3>
<ul style="color: #00008B;">
  <li>Use the <code>create_engine</code> method in Python to establish a connection to your local MySQL database</li>
</ul>


In [35]:
from sqlalchemy import create_engine
username = "root"
password = "Nagham123" 
host = "localhost"
port = "3306"
database_name = "imdb_etl" 
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}")

**<h2 style="color: #00008B;">C. Create a New Database:</h2>**

<ul style="color: #00008B;">
  <li>Write Python code to create a new database within MySQL</li>
</ul>

In [36]:
engine_with_db = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}")
merged_df.to_sql(name='imdb_movies', con=engine_with_db, if_exists='replace', index=False)
print("Data loaded successfully into 'imdb_movies' table.")

Data loaded successfully into 'imdb_movies' table.


In [37]:
pd.read_sql("SHOW COLUMNS FROM imdb_movies;", con=engine_with_db)

,Field,Type,Null,Key,Default,Extra
0,Movie_ID,text,YES,,None,
1,Name,text,YES,,None,
2,Role_Type,text,YES,,None,
3,Character_Position,text,YES,,None,
4,Title,text,YES,,None,
5,Year,int,YES,,None,
6,Rating,double,YES,,None,
7,Link,text,YES,,None,
8,Budget,double,YES,,None,
9,gross_revenue,bigint,YES,,None,


**<h2 style="color: #00008B;">D. Load the Integrated Dataset:</h2>**

<ul style="color: #00008B;">
  <li>Load the cleaned and transformed dataset into the newly created MySQL database using Python</li>
</ul>

In [38]:
query1 = """
SELECT genres, COUNT(*) AS count
FROM (
    SELECT DISTINCT title, genres
    FROM imdb_movies
) AS unique_movies
GROUP BY genres
ORDER BY count DESC
LIMIT 1;
"""

df1 = pd.read_sql(query1, con=engine_with_db)
print("Most rated:")
print(df1)

Most rated:
                                              genres  count
0  ['epic', 'gangster', 'tragedy', 'crime', 'drama']      2


In [39]:
query2 = """
SELECT title, MAX(rating) AS rating
FROM imdb_movies
GROUP BY title
ORDER BY rating DESC
LIMIT 5;
"""

df2 = pd.read_sql(query2, con=engine_with_db)
print("Top 5 Rated Movies:")
print(df2)

Top 5 Rated Movies:
                      title  rating
0  the shawshank redemption     9.3
1             the godfather     9.2
2           the dark knight     9.0
3     the godfather part ii     9.0
4              12 angry men     9.0


In [40]:
query3 = """
    SELECT Year, COUNT(*) AS movie_count
    FROM (
        SELECT DISTINCT Title, Year
        FROM imdb_movies
    ) AS unique_movies
    GROUP BY Year
    ORDER BY Year;
"""

df3 = pd.read_sql(query3, con=engine_with_db)
print("Number of unique films per year:")
print(df3)

Number of unique films per year:
    Year  movie_count
0   1946            1
1   1954            1
2   1957            1
3   1966            1
4   1972            1
5   1974            1
6   1975            1
7   1980            1
8   1990            1
9   1991            1
10  1993            1
11  1994            3
12  1995            1
13  1998            1
14  1999            2
15  2001            1
16  2002            2
17  2003            1
18  2008            1
19  2010            1
20  2014            1


**<h2 style="color: #00008B;">THE END...</h2>**